In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from persist.persist import PersistentDAG

from persist.persist import PersistentDAG
from functools import wraps

# global variable to simulate the fact to have serialize data somewhere
global IS_COMPUTED
IS_COMPUTED = dict()

from time import sleep
def load_data(*args, **kwargs):
    sleep(2)
    print 'load data ...'
    if args:
        print args
        return 'data_{}'.format(args)
    if kwargs:
        print kwargs
        return 'data_{}'.format(kwargs)
    return 'data'


def clean_data(data, *args, **kwargs):
    assert isinstance(data, str)
    print 'clean data ...'
    if args:
        print args
        data = data + '_' + '_'.join(map(lambda x: '{}'.format(x), args))
    if kwargs:
        print kwargs
        data = data + '_' + \
            '_'.join(map(lambda kv: '{}_{}'.format(
                kv[0], kv[1]), kwargs.items()))
    return 'cleaned_{}'.format(data)


def analyze_data(cleaned_data, option=1, **other_options):
    assert isinstance(cleaned_data, str)
    print 'analyze data ...'
    return 'analyzed_{}'.format(cleaned_data)



class Serializer(object):

    def __init__(self):
        pass

    def load(self, key):
        print "load data for key {} ...".format(key)
        return IS_COMPUTED[key]

    def dump(self, key, value):
        print "save data with key {} ...".format(key)
        IS_COMPUTED[key] = value

    def is_computed(self, key):
        global IS_COMPUTED
        return IS_COMPUTED.get(key) is not None

    def delayed_load(self, key):
        def load():
            return self.load(key)
        return load

    def dump_result(self, func, key):
        @wraps(func)
        def wrapped_func(*args, **kwargs):
            result = func(*args, **kwargs)
            self.dump(key, result)
            return result
        return wrapped_func

ImportError: No module named httplib

In [ ]:
def setup_graph():
    g = PersistentDAG()
    serializer = Serializer()
    g.add_task(load_data, option=10,
               dask_key_name='key_data1', 
               dask_serializer=serializer 
              )
    g.add_task(load_data, option=20,
               dask_key_name='key_data2', 
               dask_serializer=serializer
              )

    kwargs = dict(data='key_data1', other='key_data2')
    g.add_task(clean_data,
               dask_key_name='cleaned_data', 
               #dask_serializer=serializer,
               **kwargs)
    g.add_task(analyze_data, 'cleaned_data',
               # dask_key_name='analyzed_data', 
              dask_serializer=serializer)
    return g

In [ ]:
g = setup_graph()

In [ ]:
from dask.dot import dot_graph
dot_graph(g._dask, format="svg")

In [ ]:
g.visualize(format='svg', raw_dask=True)

In [ ]:
g.visualize(format='svg')

In [ ]:
g.dask

In [ ]:
IS_COMPUTED

In [ ]:
g.compute()

In [ ]:
%pylab inline

In [ ]:
g.visualize(format='svg')

In [ ]:
g.dask

In [ ]:
IS_COMPUTED

In [ ]:
g.compute()

If we instantiate another graph, it automatically detect that some data has already been computed.

In [ ]:
g = setup_graph()

In [ ]:
g.visualize(format='svg')

We can visualize the graph with methods replaced by load methods on disk.

In [ ]:
g.visualize(format='svg', raw_dask=False)

## Persist method 

In [ ]:
# g.clean()

In [ ]:
data = g.persist()

In [ ]:
data

In [ ]:
g.run()

In [ ]:
data.compute()

In [ ]:
IS_COMPUTED

In [ ]:
g.visualize(format='svg')

In [ ]:
data = g.submit(analyze_data, 'cleaned_data')
data.compute()

In [ ]:
import dask
with dask.set_options(get=dask.async.get_sync):
    g.run()

In [ ]:
IS_COMPUTED

In [ ]:
%pylab inline

In [ ]:
graph = g.to_digraph()
import networkx as nx
nx.draw(graph)

In [ ]:
from distributed import Client
client = Client()
global IS_COMPUTED
IS_COMPUTED = {}

In [ ]:
g.persist()

In [ ]:
g.compute()

In [ ]:
g.visualize(format='svg')

In [ ]:
# TODO: we have to use a real serializer in order to make things work with futures.